In [69]:
import numpy as np 
import pandas as pd
import calendar
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [70]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))
    
#Uno los dataFrame
metadata = pd.DataFrame()
for dataframe in dataframes:
    metadata = pd.concat([metadata, dataframe]).drop_duplicates().reset_index(drop=True)

In [71]:
data = metadata.loc[(metadata.state_name.str.contains('Buenos Aires')) | (metadata.state_name.str.contains\
                ('Capital Federal'))  | (metadata.state_name.str.contains\
                ('Bs.As') ),:]
data.shape
# 1.283.741 entries antes, ahora 242290

(242290, 27)

<br><B><FONT FACE="Arial" SIZE="5">Preparamos los datos: en primer lugar trabajo con el dataset que está completo y usaremos para predecir</FONT></B><br/>

In [72]:
data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'lat', 'lon', 'created_on', 'property_type']].dropna()

# filtro de datos anómalos
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)) \
               , :]


data_float.shape
#(104604, 7)

(104604, 7)

In [73]:
data_float['fecha'] = pd.to_datetime(data_float['created_on'])
data_float['anio'] = data_float['fecha'].map(lambda x:x.year)
data_float['mes'] = data_float['fecha'].map(lambda x:x.month)


In [74]:
data_visualiz = data.loc[:,['property_type','id']].groupby('property_type').agg(np.size)
data_visualiz.head(10)

,id
property_type,
PH,15012
apartment,137094
house,82560
store,7624


In [75]:
def tipoDePropiedadData(x):
    if (x== 'house'): return 1
    if (x== 'PH'): return 2
    if (x== 'apartment'): return 3
    if (x== 'store'): return 4

data_float['tipo_propiedad'] = data_float['property_type'].apply(tipoDePropiedadData)
data_float['tipo_propiedad'].describe()

count    104604.000000
mean          2.390224
std           0.932952
min           1.000000
25%           1.000000
50%           3.000000
75%           3.000000
max           4.000000
Name: tipo_propiedad, dtype: float64

In [76]:
data['property_type'].tail(10)

296467        house
296468    apartment
296469        house
296470        house
296471        store
296472    apartment
296473        house
296474    apartment
296475    apartment
296476    apartment
Name: property_type, dtype: object

### Ahora trabajamos con el set de datos cuya columna 'price_usd' hay que predecir

In [77]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test_visualiz = test.loc[:,['state_name', 'id']].groupby('state_name').agg(np.size)
test_visualiz.head(10)

,id
state_name,
Bs.As. G.B.A. Zona Norte,2
Bs.As. G.B.A. Zona Oeste,3101
Bs.As. G.B.A. Zona Sur,2215
Buenos Aires Interior,1
Capital Federal,8847


In [78]:
# Depuración del set de test
#Completo los valores NaN con promedios para realizar la predicción
prom_surface_total_in_m2 = test['surface_total_in_m2'].mean()
prom_surface_covered_in_m2 = test['surface_covered_in_m2'].mean()
prom_lat = test['lat'].mean()
prom_lon = test['lon'].mean()

test_float = test
test_float['surface_total_in_m2'].fillna(prom_surface_total_in_m2, inplace=True)
test_float['surface_covered_in_m2'].fillna(prom_surface_covered_in_m2, inplace=True)
test_float['lat'].fillna(prom_lat, inplace=True)
test_float['lon'].fillna(prom_lon, inplace=True)

test_float['fecha'] = pd.to_datetime(test_float['created_on'])
test_float['anio'] = test_float['fecha'].map(lambda x:x.year)
test_float['mes'] = test_float['fecha'].map(lambda x:x.month)

test_float.head(100)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,fecha,anio,mes
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.000000,359.471588,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,2017-08-24,2017,8
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,-58.465820,0.000000,359.471588,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...,2017-08-25,2017,8
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.000000,48.000000,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...,2017-08-01,2017,8
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,51.000000,51.000000,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...,2017-08-01,2017,8
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.000000,67.000000,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...,2017-08-01,2017,8
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,47.000000,47.000000,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...,2017-08-01,2017,8


In [79]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                       1368
rooms                       7500
expenses                    2543
description                14166
fecha                      14166
anio                       14166
mes                        14166
dtype: int64

In [80]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test_visualiz = test.loc[:,['property_type', 'id']].groupby('property_type').agg(np.size)
test_visualiz.head(10)

,id
property_type,
casa,3409
departamento,9761
ph,996


In [81]:
def tipoDePropiedadTest(x):
    if (x== 'casa'): return 1
    if (x== 'ph'): return 2
    if (x== 'departamento'): return 3
    if (x== 'negocio'): return 4

test_float['tipo_propiedad'] = test_float['property_type'].apply(tipoDePropiedadTest)
test_float['tipo_propiedad'].describe()

count    14166.000000
mean         2.448398
std          0.853629
min          1.000000
25%          2.000000
50%          3.000000
75%          3.000000
max          3.000000
Name: tipo_propiedad, dtype: float64

## Uso PCA como reductor de dimensiones para sumar dos nuevas dimensiones al dataset.
#### Para ello utilizamos las features 'surface_total_in_m2', 'surface_covered_in_m2', 'lat' y 'lon'

In [82]:
data_float_pca = data_float.loc[:,['surface_total_in_m2', 'surface_covered_in_m2',\
                          'lat', 'lon']]
data_float_pca

,surface_total_in_m2,surface_covered_in_m2,lat,lon
0,500.0,500.0,-34.643701,-58.651548
5,280.0,100.0,-34.536689,-58.566976
9,45.0,42.0,-34.603723,-58.381593
13,135.0,135.0,-34.546433,-58.545713
14,330.0,310.0,-34.345731,-58.855461
15,0.0,218.0,-34.735512,-58.398874
16,30.0,30.0,-34.649843,-58.657190
17,47.0,47.0,-34.651906,-58.644602
18,0.0,136.0,-34.797334,-58.395871
19,294.0,100.0,-34.767167,-58.215465


In [83]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(data_float_pca)
red = pca.transform(data_float_pca)
print(pca.singular_values_)

[ 284133.05197835   90861.62369182]


In [84]:
red

array([[ 361.68908677,  340.42935535],
       [  91.23008934,  -27.33677514],
       [-149.33948855,  -54.08148562],
       ..., 
       [-100.72495834,  -37.29972439],
       [-150.7235221 ,  -56.9248006 ],
       [-148.74072438,  -57.18655313]])

In [85]:
d = {'col1': red[:,0], 'col2': red[:,1]}
red_data = pd.DataFrame(data=d)

data_float.reset_index(inplace=True)
data_float['PCA1'] = red_data['col1']
data_float['PCA2'] = red_data['col2']
data_float.tail()

,index,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,lat,lon,created_on,property_type,fecha,anio,mes,tipo_propiedad,PCA1,PCA2
104599,296382,210000.0,403.0,403.0,-34.637014,-58.433177,2017-08-01,store,2017-08-01,2017,8,4,252.828236,256.958900
104600,296388,325000.0,113.0,113.0,-34.590628,-58.397539,2017-08-01,apartment,2017-08-01,2017,8,3,-72.632032,7.408077
104601,296410,165000.0,91.0,65.0,-34.599214,-58.402519,2017-08-01,apartment,2017-08-01,2017,8,3,-100.724958,-37.299724
104602,296415,165000.0,44.0,39.0,-34.587425,-58.397372,2017-08-01,apartment,2017-08-01,2017,8,3,-150.723522,-56.924801
104603,296474,131500.0,46.0,39.0,-34.570639,-58.475596,2017-08-01,apartment,2017-08-01,2017,8,3,-148.740724,-57.186553


In [86]:
test_float_pca = test_float.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                    'lat', 'lon']]
test_float_pca

,surface_total_in_m2,surface_covered_in_m2,lat,lon
0,0.0,359.471588,-34.610988,-58.363464
1,0.0,359.471588,-34.629923,-58.465820
2,53.0,48.000000,-34.589363,-58.412880
3,39.0,39.000000,-34.629923,-58.465820
4,51.0,51.000000,-34.629923,-58.465820
5,39.0,39.000000,-34.629923,-58.465820
6,39.0,39.000000,-34.629923,-58.465820
7,39.0,39.000000,-34.629923,-58.465820
8,67.0,67.000000,-34.578721,-58.457095
9,47.0,47.000000,-34.629923,-58.465820


In [87]:
red_test = pca.transform(test_float_pca)
red_test

array([[-152.40226211,  266.54887287],
       [-152.40226019,  266.5488747 ],
       [-140.6230308 ,  -49.18011945],
       ..., 
       [  35.33552885,   15.34572404],
       [-131.70044467,  -50.35802059],
       [ 181.40810645,   -8.98125454]])

In [88]:
d = {'col1': red_test[:,0], 'col2': red_test[:,1]}
red_test_data = pd.DataFrame(data=d)

#test_float.reset_index(inplace=True)
test_float['PCA1'] = red_test_data['col1']
test_float['PCA2'] = red_test_data['col2']
test_float.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,...,floor,rooms,expenses,description,fecha,anio,mes,tipo_propiedad,PCA1,PCA2
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,...,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,2017-08-24,2017,8,3,-152.402262,266.548873
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,...,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...,2017-08-25,2017,8,3,-152.402260,266.548875
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,...,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...,2017-08-01,2017,8,3,-140.623031,-49.180119
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,...,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8,3,-155.680512,-56.270410
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,...,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...,2017-08-01,2017,8,3,-142.213191,-45.944169


<br><B><FONT FACE="Arial" SIZE="5">Predicción usando GBR</FONT></B><br/>

In [89]:
l = len(data_float)
limite = 70*l/100
print l
print limite

104604
73222


In [90]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'lat', 'lon', 'PCA1', 'PCA2', 'mes', 'tipo_propiedad']
TRAIN = data_float[:limite]
TEST = data_float[limite:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [91]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [92]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.704/0.557

Precisión Gradient Boosting train/test  0.717/0.569


<br><B><FONT FACE="Arial" SIZE="5">Uso grid-search para estimar los parametros</FONT></B><br/>

In [93]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2',  'lat', 'lon', 'PCA1', 'PCA2', 'mes', 'tipo_propiedad']
TRAIN = data_float[:limite]
TEST = data_float[limite:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd

y_test = TEST.price_aprox_usd



In [48]:
# learning_rate = 0.16
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

#param_test0 = {'learning_rate': np.arange(0.01,0.5,0.01)}
#gsearch0 = GridSearchCV(estimator = GradientBoostingRegressor(n_estimators=500, min_samples_split=500,\
#min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
#param_grid = param_test0, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch0.fit(X_train, y_train)
#gsearch0.best_params_

{'learning_rate': 0.24000000000000002}

In [47]:
print np.arange(0.01,0.51,0.01)

[ 0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12
  0.13  0.14  0.15  0.16  0.17  0.18  0.19  0.2   0.21  0.22  0.23  0.24
  0.25  0.26  0.27  0.28  0.29  0.3   0.31  0.32  0.33  0.34  0.35  0.36
  0.37  0.38  0.39  0.4   0.41  0.42  0.43  0.44  0.45  0.46  0.47  0.48
  0.49  0.5 ]


In [49]:
# n_estimators = 591
#param_test1 = {'n_estimators':list(range(1,600,10))}

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.24, min_samples_split=500,\
#min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
#param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1 ,max_features='sqrt',\
#subsample=0.8, random_state=10), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
                        
#gsearch1.fit(X_train, y_train)
#Me fijo el resultado del parametro
#gsearch1.best_params_

{'n_estimators': 591}

In [ ]:
#'max_depth': 12, 'min_samples_split': 240

#param_test2 = {'max_depth':list(range(2,16,2)), 'min_samples_split':list(range(200,1001,10))}
#gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.24, n_estimators=591, max_features='sqrt', subsample=0.8, random_state=10), 
#param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
#gsearch2.fit(X_train,y_train)
#gsearch2.best_params_

In [53]:
#'min_samples_leaf': 19

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

#param_test3 = {'min_samples_leaf':list(range(10,100,1))}
#gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.24, n_estimators=591, min_samples_split=240, max_depth=10,max_features='sqrt', subsample=0.8, random_state=10), 
#param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
#gsearch3.fit(X_train, y_train)
#gsearch3.best_params_

{'min_samples_leaf': 35}

In [56]:
#'max_features': 2

#param_test4 = {'max_features':list(range(1,6,1))}
#gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.24, n_estimators=591, max_depth=10, min_samples_split=240, min_samples_leaf=35
#                                                              , subsample=0.8, random_state=10),
#param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
#gsearch4.fit(X_train, y_train)
#gsearch4.best_params_

{'max_features': 2}

In [94]:
# 'n_estimators': 591, 'max_depth': 12, 'min_samples_split': 240, 'min_samples_leaf': 19, 'max_features':2

from sklearn import ensemble
params = {'n_estimators': 591, 'max_depth': 12, 'min_samples_split': 240, 'min_samples_leaf':19 , 'max_features': 2,
          'learning_rate': 0.24}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.907/0.603
#Precisión Gradient Boosting train/test  0.925/0.626

Precisión Gradient Boosting train/test  0.926/0.618


In [95]:
#Hago la prediccion para Kaggle
test_float['price_aprox_usd_XGB'] = clf.predict(test_float.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                         'lat', 'lon', 'PCA1', 'PCA2', 'mes', 'tipo_propiedad']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS.csv', index=False)

In [96]:
# Imprimo la relevancia de cada feature para el regresor
# feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2',  'lat', 'lon', 'PCA1', 'PCA2', 'mes', 'tipo_propiedad]

print clf.feature_importances_

[ 0.11640548  0.21221503  0.11460264  0.11410096  0.15468282  0.25688497
  0.01640278  0.01470532]
